In [107]:
from datasets import load_dataset
import pandas as pd
import requests
import numpy
import matplotlib.pyplot as plt
from IPython.display import Audio
import matplotlib.pyplot as plt
from IPython.display import Audio

In [113]:
dataset = load_dataset("autumnjohnson/ceti_audio")

In [114]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'coda_type', 'path', 'sampling_rate'],
        num_rows: 3176
    })
    test: Dataset({
        features: ['audio', 'coda_type', 'path', 'sampling_rate'],
        num_rows: 353
    })
})

In [118]:
dataset['train']

Dataset({
    features: ['audio', 'coda_type', 'path', 'sampling_rate'],
    num_rows: 3176
})

In [119]:
train_label = dataset['train']['coda_type']

In [120]:
train_label

['10i',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '5R3',
 '1+1+3',
 '1+1+3',
 '5R2',
 '1+1+3',
 '3D',
 '4R2',
 '5R1',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '5R1',
 '8i',
 '1+1+3',
 '5R3',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '4R2',
 '3D',
 '1+1+3',
 '1+1+3',
 '5R1',
 '5R1',
 '4R2',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '4R1',
 '2+3',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '10i',
 '1+1+3',
 '1+1+3',
 '4R2',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '5R1',
 '1+1+3',
 '1+1+3',
 '5R1',
 '5R1',
 '1+1+3',
 '1+1+3',
 '4R2',
 '1+1+3',
 '1+1+3',
 '5R2',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '5R1',
 '3D',
 '5R1',
 '1+1+3',
 '1+1+3',
 '3D',
 '5R1',
 '5R1',
 '7D1',
 '5R3',
 '5R1',
 '5R1',
 '1+1+3',
 '8i',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '2+3',
 '1+1+3',
 '5R2',
 '5R1',
 '5R3',
 '5R1',
 '1+1+3',
 '8i',
 '1+1+3',
 '5R1',
 '1+1+3',
 '8i',
 '1+1+3',
 '5R1',
 '1+1+3',
 '5R3',
 '1+1+3',
 '1+1+3',
 '7D2',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '1+1+3',
 '4R2',
 '1+1+3',
 '1+1+3',
 '8R',
 '1

In [130]:
train_audio = dataset['train']['audio']

In [129]:
train_audio

array([-0.03286743, -0.06503296, -0.05688477, ..., -0.05682373,
       -0.06509399,  0.        ])